## Визуализация работы FAISS

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import sys
import json
# Получаем абсолютный путь к корневой директории проекта (директория выше текущей)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Добавляем корневую директорию в sys.path
if root_path not in sys.path:
    sys.path.append(root_path)

In [6]:
import torch
from utils import parse_yaml
from models.clap_encoder import CLAP_Encoder
import faiss
import numpy as np

2024-05-06 12:28:37,011 - INFO - Loading faiss with AVX2 support.
2024-05-06 12:28:37,026 - INFO - Successfully loaded faiss with AVX2 support.


In [7]:
SS_CONFIG_PATH = '../config/audiosep_base.yaml'
CLAP_CKPT_PATH = '../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'

In [8]:
device = torch.device('cuda')
configs = parse_yaml(SS_CONFIG_PATH)

query_encoder = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)

2024-05-06 12:28:37,291 - INFO - Loading HTSAT-base model config.
2024-05-06 12:28:42,325 - INFO - Loading pretrained HTSAT-base-roberta weights (../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt).


In [9]:
# Эмбеддинги, которые сохраняем в бд
saved_classes = ['vocal', 'drums', 'guitar', 'hippopotamus', 'roar', 'blender']
# Запросы, к которым будем искать ближайший класс из бд
query_classes = ['kick', 'ukulele', 'singing', 'howl', 'scream']

In [10]:
embeddings_to_save = query_encoder.get_query_embed(modality='text', text=saved_classes).cpu()
embeddings_to_query = query_encoder.get_query_embed(modality='text', text=query_classes).cpu()
embeddings_to_save.shape, embeddings_to_query.shape

(torch.Size([6, 512]), torch.Size([5, 512]))

In [11]:
index = faiss.IndexHNSWFlat(512, 32)
index.add(embeddings_to_save)
distances, indices = index.search(embeddings_to_query, k = len(saved_classes))

In [12]:
distances

array([[1.0701145 , 1.3757946 , 1.4211205 , 1.4345753 , 1.5783528 ,
        1.6412584 ],
       [1.1907144 , 1.3707284 , 1.3740044 , 1.4348282 , 1.8569329 ,
        1.8919489 ],
       [0.5037271 , 0.9429968 , 0.98835295, 1.4366088 , 1.5765313 ,
        1.9686406 ],
       [0.9751475 , 0.9951929 , 1.2816312 , 1.4155016 , 1.5595326 ,
        1.8042557 ],
       [1.0844126 , 1.0952895 , 1.2217162 , 1.3817437 , 1.5443419 ,
        1.5597425 ]], dtype=float32)

In [13]:
indices

array([[1, 0, 2, 4, 3, 5],
       [2, 0, 3, 1, 5, 4],
       [0, 1, 3, 2, 5, 4],
       [3, 0, 1, 2, 4, 5],
       [1, 3, 0, 2, 5, 4]])

In [14]:
for i, query_class in enumerate(query_classes):
    k_nearest_indices = indices[i]
    k_nearest_distances = distances[i]
    class_distance_tuples = [
        (saved_classes[class_index], distance) for class_index, distance in zip(k_nearest_indices, k_nearest_distances)]

    k_nearest_names = [f'{cls}__{dist:.2f}' for (cls, dist) in class_distance_tuples]
    difference_between_top2 = class_distance_tuples[1][1] - class_distance_tuples[0][1]
    difference_between_last = class_distance_tuples[len(k_nearest_distances) - 1][1] - class_distance_tuples[0][1]

    print(f'Class {query_class}, '
          f'nearest classes: {k_nearest_names}, '
          f'difference between top 1 and top2: {difference_between_top2} '
          f'difference between top 1 and last: {difference_between_last}')

Class kick, nearest classes: ['drums__1.07', 'vocal__1.38', 'guitar__1.42', 'roar__1.43', 'hippopotamus__1.58', 'blender__1.64'], difference between top 1 and top2: 0.30568015575408936 difference between top 1 and last: 0.5711438655853271
Class ukulele, nearest classes: ['guitar__1.19', 'vocal__1.37', 'hippopotamus__1.37', 'drums__1.43', 'blender__1.86', 'roar__1.89'], difference between top 1 and top2: 0.1800140142440796 difference between top 1 and last: 0.7012345790863037
Class singing, nearest classes: ['vocal__0.50', 'drums__0.94', 'hippopotamus__0.99', 'guitar__1.44', 'blender__1.58', 'roar__1.97'], difference between top 1 and top2: 0.4392697215080261 difference between top 1 and last: 1.4649134874343872
Class howl, nearest classes: ['hippopotamus__0.98', 'vocal__1.00', 'drums__1.28', 'guitar__1.42', 'roar__1.56', 'blender__1.80'], difference between top 1 and top2: 0.02004539966583252 difference between top 1 and last: 0.8291082382202148
Class scream, nearest classes: ['drums__

In [15]:
np.square(embeddings_to_save[0]).sum()

tensor(1.0000)

Вектора нормализованы, по неравенству треугольника максимальное расстояние между векторами = 2

Видно, что энкодер правильно нашел ближайший класс для классов kick, ukulele и singing.

Однако для классов howl и scream энкодер не нашел класс roar, являющийся очевидным синонимом. Возможно такого класса не было в словаре у энкодера.

Также отметим, что для каждого класса из запроса имелся только один правильный класс, сохраненный в базе данных, однако разница между топ 1 и топ 2 классами для неправильно определенных классов и для класса vocal оказалась незначительной, что может говорить о несовершенстве энкодера.

Также для самого близкого класса singing ближайший класс vocal лежит на расстоянии 0.5 (L2 norm). Это намного ближе относительно второго места, однако если посмотреть на первые места сложно определить правильный threshold (когда использовать адаптер, а когда базовую модель) для синонимичных классов.

Однако исходя из того, что CLAP является SOTA решением для определения близости captions в контексте звука, решено использовать в качестве эмбеддера именно его.

# Также интересно проанализировать близость запросов к классам audioset

In [16]:
audioset_classes_path = 'ontology.json'
with open(audioset_classes_path, 'r') as f:
    data = json.load(f)
    names = [x for x in map(lambda x: x['name'], data)]
    # добавляем lower-cased названия, дальше будет описано зачем
    names += [x.lower() for x in map(lambda x: x['name'], data)]


In [17]:
embeddings_to_save = np.asarray([query_encoder.get_query_embed(modality='text', text=[x]).cpu() for x in names]).squeeze(1)

In [18]:
index = faiss.IndexHNSWFlat(512, 32)
index.add(np.asarray(embeddings_to_save))
distances, indices = index.search(embeddings_to_query, k = 3)

In [26]:
def overview_query_result(query_classes, indices, distances):
    for i, query_class in enumerate(query_classes):
        k_nearest_indices = indices[i]
        k_nearest_distances = distances[i]
        class_distance_tuples = [
            (names[class_index], distance) for class_index, distance in zip(k_nearest_indices, k_nearest_distances)]

        k_nearest_names = [f'{cls}__{dist:.2f}' for (cls, dist) in class_distance_tuples]
        difference_between_top2 = class_distance_tuples[1][1] - class_distance_tuples[0][1]
        difference_between_last = class_distance_tuples[len(k_nearest_distances) - 1][1] - class_distance_tuples[0][1]

        print(f'Class {query_class}, '
              f'nearest classes: {k_nearest_names}, '
              f'difference between top 1 and top2: {difference_between_top2} '
              f'difference between top 1 and last: {difference_between_last}')

overview_query_result(query_classes, indices, distances)

Class kick, nearest classes: ['thump, thud__0.79', 'whack, thwack__0.84', 'Thump, thud__0.86'], difference between top 1 and top2: 0.04906284809112549 difference between top 1 and last: 0.06486690044403076
Class ukulele, nearest classes: ['ukulele__0.00', 'Ukulele__0.19', 'Mandolin__0.68'], difference between top 1 and top2: 0.1898217350244522 difference between top 1 and last: 0.6820544004440308
Class singing, nearest classes: ['singing__0.00', 'Singing__0.18', 'vocal music__0.47'], difference between top 1 and top2: 0.18427009880542755 difference between top 1 and last: 0.47043418884277344
Class howl, nearest classes: ['howl__0.00', 'yawn__0.48', 'hoot__0.59'], difference between top 1 and top2: 0.4791603088378906 difference between top 1 and last: 0.5930732488632202
Class scream, nearest classes: ['screaming__0.37', 'battle cry__0.44', 'yell__0.49'], difference between top 1 and top2: 0.07813376188278198 difference between top 1 and last: 0.12256881594657898


Замечаем, что CLAP - case sensitive (ukulele и Ukulele - разные вектора) - видимо при его обучении captions не приводили к нижнему регистру.

Также видим, что если пространство классов большое - качество top-k классификации хорошее.

## Сравним ближайшие классы из musdb для базовой модели AudioSep и лучшей embeddings модели

In [21]:
from models.audiosep_tunned_embeddings import AudioSepTunedEmbeddings
from model_loaders import load_ss_model

SS_CONFIG_PATH = '../config/audiosep_base.yaml'
CLAP_CKPT_PATH = '../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'
AUDIOSEP_CKPT_PATH = '../checkpoint/audiosep_base_4M_steps.ckpt'
device = torch.device('cuda')
configs = parse_yaml(SS_CONFIG_PATH)

checkpoint_path = '../checkpoints/final/musdb18/embeddings/final.ckpt'

query_encoder_for_lora = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)
base_model_for_lora = load_ss_model(configs=configs, checkpoint_path=AUDIOSEP_CKPT_PATH, query_encoder=query_encoder_for_lora).eval().to(device)

model = AudioSepTunedEmbeddings.load_from_checkpoint(
    checkpoint_path=checkpoint_path,
    strict=False,
    ss_model=base_model_for_lora.ss_model,
    query_encoder=base_model_for_lora.query_encoder,
    waveform_mixer=None,
    loss_function=None,
    optimizer_type=None,
    learning_rate=None,
    lr_lambda_func=None,
) \
    .eval() \
    .to(device)


2024-05-06 12:31:17,874 - INFO - Loading HTSAT-base model config.
2024-05-06 12:31:19,174 - INFO - Loading pretrained HTSAT-base-roberta weights (../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt).


In [37]:
classes = ["bass", "drums", "vocals", 'other musical instruments']

base_embeddings = np.asarray([query_encoder.get_query_embed(modality='text', text=[x]).cpu() for x in classes]).squeeze(1)

tuned_embeddings = np.asarray([model.query_encoder.get_query_embed(modality='text', text=[x]).detach().cpu() for x in classes]).squeeze(1)

base_distances, base_indices = index.search(base_embeddings, k = 3)
tuned_distances, tuned_indices = index.search(tuned_embeddings, k = 3)

overview_query_result(classes, base_indices, base_distances)

Class bass, nearest classes: ['chord__0.46', 'recording__0.47', 'zing__0.47'], difference between top 1 and top2: 0.013088047504425049 difference between top 1 and last: 0.017460674047470093
Class drums, nearest classes: ['drum__0.23', 'music__0.35', 'chord__0.35'], difference between top 1 and top2: 0.11751864850521088 difference between top 1 and last: 0.12226851284503937
Class vocals, nearest classes: ['bellow__0.41', 'music role__0.46', 'music__0.48'], difference between top 1 and top2: 0.05037081241607666 difference between top 1 and last: 0.06213861703872681
Class other musical instruments, nearest classes: ['music mood__0.35', 'exciting music__0.46', 'dance music__0.50'], difference between top 1 and top2: 0.11006090044975281 difference between top 1 and last: 0.1472373604774475


In [38]:
overview_query_result(classes, tuned_indices, tuned_distances)

Class bass, nearest classes: ['Bass guitar__14.33', 'Bathtub (filling or washing)__14.42', 'pizzicato__14.44'], difference between top 1 and top2: 0.09071826934814453 difference between top 1 and last: 0.11366748809814453
Class drums, nearest classes: ['cymbal__10.32', 'crash cymbal__10.38', 'Crash cymbal__10.38'], difference between top 1 and top2: 0.05514717102050781 difference between top 1 and last: 0.060451507568359375
Class vocals, nearest classes: ['choir__7.14', 'chant__7.31', 'sigh__7.31'], difference between top 1 and top2: 0.17082548141479492 difference between top 1 and last: 0.17502498626708984
Class other musical instruments, nearest classes: ['choir__11.66', 'Civil defense siren__11.91', 'opera__11.98'], difference between top 1 and top2: 0.24882984161376953 difference between top 1 and last: 0.3181324005126953


Видим, что классы становятся ближе к похожим на них описаниям. С другой стороны, вектор для класс drums ближе к классу cymbal (тарелки барабанов), чем к существующему в audioset классу drums. Возможно, это эффект переобучения - очень низкие частоты модель определяет в класс bass, в то время как более высокие - в класс drums. Это частично подтверждается эмперически - цокоющие высокие звуки вокала / других музыкальных инструментов модель добавляет в аудиозапись класса drums.

Также заетим, что затюненые вектора перестают быть нормализованными.

In [43]:
display([np.linalg.norm(x) for x in base_embeddings])
display([np.linalg.norm(x) for x in tuned_embeddings])

[0.99999994, 1.0, 1.0, 1.0]

[3.8561718, 3.263241, 2.6703308, 3.4665463]

## Посмотрим, не превращает ли энкодер неизвестные слова в одинаковые вектора

In [14]:
a = query_encoder.get_query_embed(modality='text', text=['wioewjmfoermfp weofkmqew']).cpu()
b = query_encoder.get_query_embed(modality='text', text=['dvfdfdbsfbs dasd']).cpu()
np.linalg.norm(a-b)

0.70498794

Не превращает, поэтому даже неизвестные caption можно использовать при дообучении